In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
from efficientnet_pytorch import EfficientNet
from pathlib import Path
import numpy as np
import multiprocessing

In [2]:
# check if cuda is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Model

In [3]:
v = 0     # model version
in_c = 2  # number of input channels
num_c = 1 # number of classes to predict

In [4]:
# The optical flow input will look like this
of = torch.randn(1,2,640,480)

In [5]:
model = EfficientNet.from_pretrained(f'efficientnet-b{v}', in_channels=in_c, num_classes=num_c)
model.to(device);

Loaded pretrained weights for efficientnet-b0


#### The output of the model will look like this

In [6]:
of = of.to(device)
model(of).item()

0.08474025130271912

## Data

In [7]:
# directory with the optical flow images
of_dir = '<YOUR DIRECTORY>'
# labels as txt file
labels_f = '<YOUR FILE>'

In [8]:
class OFDataset(Dataset):
    def __init__(self, of_dir, label_f):
        self.len = len(list(Path(of_dir).glob('*.npy')))
        self.of_dir = of_dir
        self.label_file = open(label_f).readlines()
    def __len__(self): return self.len
    def __getitem__(self, idx):
        of_array = np.load(Path(self.of_dir)/f'{idx}.npy')
        of_tensor = torch.squeeze(torch.Tensor(of_array))
        label = float(self.label_file[idx].split()[0])
        return [of_tensor, label]

In [9]:
ds = OFDataset(of_dir, labels_f)

In [10]:
# 80% of data for training
# 20% of data for validation
train_split = .8

In [13]:
ds_size = len(ds)
indices = list(range(ds_size))
split = int(np.floor(train_split * ds_size))
train_idx, val_idx = indices[:split], indices[split:]

In [14]:
sample = ds[3]
assert type(sample[0]) == torch.Tensor
assert type(sample[1]) == float

In [15]:
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [16]:
cpu_cores = multiprocessing.cpu_count()
cpu_cores

6

In [17]:
train_dl = DataLoader(ds, batch_size=8, sampler=train_sampler, num_workers=cpu_cores)
val_dl = DataLoader(ds, batch_size=8, sampler=val_sampler, num_workers=cpu_cores)

## Train

In [18]:
epochs = 1000
log_train_steps = 100

In [19]:
criterion = nn.MSELoss()
opt = optim.Adam(model.parameters())

In [ ]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, sample in enumerate(train_dl):
        of_tensor = sample[0].cuda()
        label = sample[1].float().cuda()
        opt.zero_grad()
        pred = torch.squeeze(model(of_tensor))
        loss = criterion(pred, label)
        loss.backward()
        opt.step()
    # validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for j, val_sample in enumerate(val_dl):
            of_tensor = val_sample[0].cuda()
            label = val_sample[1].float().cuda()
            pred = torch.squeeze(model(of_tensor))
            loss = criterion(pred, label)
            val_losses.append(loss)
        print(f'{epoch}: {sum(val_losses)/len(val_losses)}')